In [ ]:
# Team  member 1 SRN and name: PES1UG22AM069 Hiya Jain
# Team  member 2 SRN and name: PES1UG22AM141 Sahana Patil
# Team  member 3 SRN and name: PES1UG22AM153 Shreya Umesh Sadalgekar
# Team  member 4 SRN and name: PES1UG22AM910 Aastha Kumari


In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes -q
!pip install trl -q  # optional, if you're using SFT or RLHF later


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# ✅ 1. Import libraries
from datasets import load_dataset
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from transformers import AutoTokenizer
import torch
import numpy as np

# Download NLTK data (fix the error)
try:
    nltk.download('punkt', quiet=True)
    nltk.download('punkt_tab', quiet=True)  # Explicitly download punkt_tab
except Exception as e:
    print(f"Error downloading NLTK data: {e}")
    exit(1)

# Verify NLTK data
try:
    word_tokenize("Test sentence.")
except LookupError:
    print("NLTK punkt/punkt_tab still not found. Please check your NLTK installation.")
    exit(1)

# ✅ 2. Load BillSum
ds = load_dataset("billsum", split=None)

# ✅ 3. Convert splits to DataFrames
train_df = pd.DataFrame(ds["train"])  # ~18,949 US bills
test_df = pd.DataFrame(ds["test"])    # ~3,269 US bills
val_df = pd.DataFrame(ds["ca_test"])  # ~1,237 CA bills (use as validation)

# ✅ 4. Confirm structure
print("Columns:", train_df.columns)  # Expected: ['text', 'summary', 'title']
print("Sample:\n", train_df[['text', 'summary', 'title']].head(5))



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.27k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/91.8M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

ca_test-00000-of-00001.parquet:   0%|          | 0.00/6.12M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

Columns: Index(['text', 'summary', 'title'], dtype='object')
Sample:
                                                 text  \
0  SECTION 1. LIABILITY OF BUSINESS ENTITIES PROV...   
1  SECTION 1. SHORT TITLE.\n\n    This Act may be...   
2  SECTION 1. SHORT TITLE.\n\n    This Act may be...   
3  SECTION 1. NONRECOGNITION OF GAIN WHERE ROLLOV...   
4  SECTION 1. SHORT TITLE.\n\n    This Act may be...   

                                             summary  \
0  Shields a business entity from civil liability...   
1  Human Rights Information Act - Requires certai...   
2  Jackie Robinson Commemorative Coin Act - Direc...   
3  Amends the Internal Revenue Code to provide (t...   
4  Native American Energy Act - (Sec. 3) Amends t...   

                                               title  
0  A bill to limit the civil liability of busines...  
1                       Human Rights Information Act  
2             Jackie Robinson Commemorative Coin Act  
3  To amend the Internal Revenue Cod

In [ ]:
# ✅ 5. Clean text
def clean_text(text):
    """Clean bill text by removing boilerplate, citations, and extra whitespace."""
    if not isinstance(text, str):  # Handle non-string inputs
        return ""
    # Remove citations (e.g., "[H.R. 1234]", "Section 2(a)")
    text = re.sub(r'\[H\.R\.\s*\d+\]|\[S\.\s*\d+\]|Section\s*\d+\([a-zA-Z]\)', '', text)
    # Remove boilerplate phrases
    boilerplate = r'(?i)(Be it enacted by the Senate and House|In the Senate of the United States|Congress finds that)'
    text = re.sub(boilerplate, '', text)
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

train_df['text'] = train_df['text'].apply(clean_text)
train_df['summary'] = train_df['summary'].apply(clean_text)
val_df['text'] = val_df['text'].apply(clean_text)
val_df['summary'] = val_df['summary'].apply(clean_text)
test_df['text'] = test_df['text'].apply(clean_text)
test_df['summary'] = test_df['summary'].apply(clean_text)



In [ ]:
# ✅ 6. Filter data
min_text_words = 100   # Minimum words for bill text
min_summary_words = 20 # Minimum words for summary
max_text_words = 5000  # Maximum words for bill text (to fit model)

def word_count(text):
    """Count words using NLTK tokenizer."""
    if not isinstance(text, str) or not text:
        return 0
    return len(word_tokenize(text))

# Filter based on length
train_df = train_df[
    (train_df['text'].apply(word_count) >= min_text_words) &
    (train_df['text'].apply(word_count) <= max_text_words) &
    (train_df['summary'].apply(word_count) >= min_summary_words)
]
print("done with train")
val_df = val_df[
    (val_df['text'].apply(word_count) >= min_text_words) &
    (val_df['text'].apply(word_count) <= max_text_words) &
    (val_df['summary'].apply(word_count) >= min_summary_words)
]
print("done with val")
test_df = test_df[
    (test_df['text'].apply(word_count) >= min_text_words) &
    (test_df['text'].apply(word_count) <= max_text_words) &
    (test_df['summary'].apply(word_count) >= min_summary_words)
]
print("done with test")

# Token-based filtering
tokenizer = AutoTokenizer.from_pretrained("t5-base", add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token

def token_count(text):
    if not isinstance(text, str) or not text:
        return 0
    return len(tokenizer.encode(text, add_special_tokens=True))

max_input_tokens = 2000
max_summary_tokens = 400
min_input_tokens = 50
min_summary_tokens = 20

train_df = train_df[
    (train_df['text'].apply(token_count) >= min_input_tokens) &
    (train_df['text'].apply(token_count) <= max_input_tokens) &
    (train_df['summary'].apply(token_count) >= min_summary_tokens) &
    (train_df['summary'].apply(token_count) <= max_summary_tokens)
]
val_df = val_df[
    (val_df['text'].apply(token_count) >= min_input_tokens) &
    (val_df['text'].apply(token_count) <= max_input_tokens) &
    (val_df['summary'].apply(token_count) >= min_summary_tokens) &
    (val_df['summary'].apply(token_count) <= max_summary_tokens)
]
test_df = test_df[
    (test_df['text'].apply(token_count) >= min_input_tokens) &
    (test_df['text'].apply(token_count) <= max_input_tokens) &
    (test_df['summary'].apply(token_count) >= min_summary_tokens) &
    (test_df['summary'].apply(token_count) <= max_summary_tokens)
]

# Remove duplicates and nulls
train_df = train_df.drop_duplicates(subset='text').dropna(subset=['text', 'summary'])
print("done with train")
val_df = val_df.drop_duplicates(subset='text').dropna(subset=['text', 'summary'])
print("done with val")
test_df = test_df.drop_duplicates(subset='text').dropna(subset=['text', 'summary'])
print("done with test")
# Re-apply sampling
train_df = train_df.sample(n=min(5000, len(train_df)), random_state=42)





done with train
done with val
done with test


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

done with train
done with val
done with test


In [ ]:
# Verify sizes
print("Train DataFrame size after filtering:", len(train_df))
print("Validation DataFrame size after filtering:", len(val_df))
print("Test DataFrame size after filtering:", len(test_df))

Train DataFrame size after filtering: 5000
Validation DataFrame size after filtering: 437
Test DataFrame size after filtering: 1759


In [ ]:

# Sample for faster training (optional, adjust based on compute)
train_df = train_df.sample(n=min(5000, len(train_df)), random_state=42)  # Reduced to 5K for speed

# ✅ 7. Format for summarization
def format_for_summarization(row):
    """Format as input-output pair for summarization."""
    input_text = f"[CLS] Summarize: {row['text']} [SEP]"
    output_text = f"{row['summary']} [EOS]"
    return pd.Series({'input_text': input_text, 'output_text': output_text})

train_df[['input_text', 'output_text']] = train_df.apply(format_for_summarization, axis=1)
val_df[['input_text', 'output_text']] = val_df.apply(format_for_summarization, axis=1)
test_df[['input_text', 'output_text']] = test_df.apply(format_for_summarization, axis=1)



In [ ]:
# Check tokenized lengths
train_df['input_text_length'] = train_df['input_text'].apply(token_count) if len(train_df) > 0 else pd.Series([])
train_df['output_text_length'] = train_df['output_text'].apply(token_count) if len(train_df) > 0 else pd.Series([])
val_df['input_text_length'] = val_df['input_text'].apply(token_count) if len(val_df) > 0 else pd.Series([])
val_df['output_text_length'] = val_df['output_text'].apply(token_count) if len(val_df) > 0 else pd.Series([])
test_df['input_text_length'] = test_df['input_text'].apply(token_count) if len(test_df) > 0 else pd.Series([])
test_df['output_text_length'] = test_df['output_text'].apply(token_count) if len(test_df) > 0 else pd.Series([])

print("\nTrain input_text length stats:")
print(train_df['input_text_length'].describe())
print("\nTrain output_text length stats:")
print(train_df['output_text_length'].describe())
print("\nValidation input_text length stats:")
print(val_df['input_text_length'].describe())
print("\nValidation output_text length stats:")
print(val_df['output_text_length'].describe())
print("\nTest input_text length stats:")
print(test_df['input_text_length'].describe())
print("\nTest output_text length stats:")
print(test_df['output_text_length'].describe())


Train input_text length stats:
count    5000.000000
mean     1417.796800
std       298.618354
min       367.000000
25%      1172.000000
50%      1386.500000
75%      1654.000000
max      2012.000000
Name: input_text_length, dtype: float64

Train output_text length stats:
count    5000.000000
mean      186.395800
std        92.753338
min        29.000000
25%       109.000000
50%       174.000000
75%       254.000000
max       404.000000
Name: output_text_length, dtype: float64

Validation input_text length stats:
count     437.000000
mean     1434.645309
std       270.241737
min       952.000000
25%      1202.000000
50%      1405.000000
75%      1628.000000
max      2006.000000
Name: input_text_length, dtype: float64

Validation output_text length stats:
count    437.000000
mean     264.954233
std       82.169397
min       41.000000
25%      207.000000
50%      271.000000
75%      328.000000
max      404.000000
Name: output_text_length, dtype: float64

Test input_text length stats:
cou

In [ ]:
from transformers import AutoTokenizer

model_name = "t5-base"  # Example: Use Mistral 7B
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token  # Ensure padding token is set
print("Pad token:", tokenizer.pad_token)
print("Pad token ID:", tokenizer.pad_token_id)
print("EOS token:", tokenizer.eos_token)
print("EOS token ID:", tokenizer.eos_token_id)

Pad token: </s>
Pad token ID: 1
EOS token: </s>
EOS token ID: 1


In [ ]:
def tokenize_data(df, tokenizer, max_length=512):
    if len(df) == 0:
        raise ValueError(f"Cannot tokenize empty DataFrame: {df.name}")
    print(f"\nTokenizing {df.name}:")
    input_texts = df['input_text'].tolist()
    output_texts = df['output_text'].tolist()

    input_lengths = [len(tokenizer.encode(text, add_special_tokens=True)) for text in input_texts]
    output_lengths = [len(tokenizer.encode(text, add_special_tokens=True)) for text in output_texts]
    print(f"Input text lengths: min={min(input_lengths)}, max={max(input_lengths)}, mean={sum(input_lengths)/len(input_lengths):.2f}")
    print(f"Output text lengths: min={min(output_lengths)}, max={max(output_lengths)}, mean={sum(output_lengths)/len(output_lengths):.2f}")

    input_encodings = tokenizer(
        input_texts,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='pt',
        add_special_tokens=True,
        return_length=True
    )
    output_encodings = tokenizer(
        output_texts,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='pt',
        add_special_tokens=True,
        return_length=True
    )

    pad_token_id = tokenizer.pad_token_id
    for i, (input_ids, input_attention_mask, input_length, output_ids, output_attention_mask) in enumerate(zip(
        input_encodings['input_ids'],
        input_encodings['attention_mask'],
        input_encodings['length'],
        output_encodings['input_ids'],
        output_encodings['attention_mask']
    )):
        num_pad_input = (input_ids == pad_token_id).sum().item()
        num_non_padding_input = input_attention_mask.sum().item()
        num_pad_output = (output_ids == pad_token_id).sum().item()
        num_non_padding_output = output_attention_mask.sum().item()
        if i < 3:
            print(f"Sample {i}:")
            print(f"  Input: Length={input_length}, Non-padding tokens={num_non_padding_input}, Pad tokens={num_pad_input}")
            print(f"  Output: Length={num_non_padding_output}, Pad tokens={num_pad_output}")
        if num_pad_input == 0 and input_lengths[i] < max_length:
            print(f"Warning: Sample {i} input has no padding despite length {input_lengths[i]}")
        if num_non_padding_input == max_length and input_lengths[i] < max_length:
            print(f"Warning: Sample {i} input attention mask has {num_non_padding_input} non-padding tokens despite length {input_lengths[i]}")
        if num_pad_output == 0 and output_lengths[i] < max_length:
            print(f"Warning: Sample {i} output has no padding despite length {output_lengths[i]}")
        if num_non_padding_output == max_length and output_lengths[i] < max_length:
            print(f"Warning: Sample {i} output attention mask has {num_non_padding_output} non-padding tokens despite length {output_lengths[i]}")

    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': output_encodings['input_ids'],
        'labels_attention_mask': output_encodings['attention_mask']
    }

# Test on subset
# train_subset = train_df.head(10)
# train_subset.name = "Train_subset"
train_df.name="Train"
val_df.name="Val"
test_df.name="Test"

try:
    # train_subset_encodings = tokenize_data(train_subset, tokenizer)
    train_encodings = tokenize_data(train_df, tokenizer)
    val_encodings = tokenize_data(val_df, tokenizer)
    test_encodings = tokenize_data(test_df, tokenizer)
except ValueError as e:
    print(e)
    exit(1)

# Inspect attention masks
def inspect_attention_mask(encodings, tokenizer, name, num_samples=3):
    print(f"\nInspecting {name} attention mask (first {num_samples} samples):")
    for i in range(min(num_samples, len(encodings['input_ids']))):
        input_ids = encodings['input_ids'][i]
        attention_mask = encodings['attention_mask'][i]
        labels = encodings['labels'][i]
        labels_attention_mask = encodings['labels_attention_mask'][i]
        decoded_input = tokenizer.decode(input_ids, skip_special_tokens=False)
        decoded_output = tokenizer.decode(labels, skip_special_tokens=False)
        num_non_padding_input = attention_mask.sum().item()
        num_non_padding_output = labels_attention_mask.sum().item()
        print(f"\nSample {i}:")
        print(f"Input attention mask unique values: {attention_mask.unique().tolist()}")
        print(f"Input non-padding tokens: {num_non_padding_input}")
        print(f"Input decoded (first 100 chars): {decoded_input[:100]} ...")
        print(f"Input pad tokens: {(input_ids == tokenizer.pad_token_id).sum().item()}")
        print(f"Output attention mask unique values: {labels_attention_mask.unique().tolist()}")
        print(f"Output non-padding tokens: {num_non_padding_output}")
        print(f"Output decoded (first 100 chars): {decoded_output[:100]} ...")
        print(f"Output pad tokens: {(labels == tokenizer.pad_token_id).sum().item()}")

# inspect_attention_mask(train_subset_encodings, tokenizer, "Train_subset")
inspect_attention_mask(train_encodings, tokenizer, "Train")
inspect_attention_mask(val_encodings, tokenizer, "Validation")
inspect_attention_mask(test_encodings, tokenizer, "Test")


Tokenizing Train:
Input text lengths: min=367, max=2012, mean=1417.80
Output text lengths: min=29, max=404, mean=186.40
Sample 0:
  Input: Length=512, Non-padding tokens=512, Pad tokens=1
  Output: Length=234, Pad tokens=279
Sample 1:
  Input: Length=512, Non-padding tokens=512, Pad tokens=1
  Output: Length=135, Pad tokens=378
Sample 2:
  Input: Length=512, Non-padding tokens=512, Pad tokens=1
  Output: Length=68, Pad tokens=445

Tokenizing Val:
Input text lengths: min=952, max=2006, mean=1434.65
Output text lengths: min=41, max=404, mean=264.95
Sample 0:
  Input: Length=512, Non-padding tokens=512, Pad tokens=1
  Output: Length=245, Pad tokens=268
Sample 1:
  Input: Length=512, Non-padding tokens=512, Pad tokens=1
  Output: Length=319, Pad tokens=194
Sample 2:
  Input: Length=512, Non-padding tokens=512, Pad tokens=1
  Output: Length=316, Pad tokens=197

Tokenizing Test:
Input text lengths: min=463, max=2011, mean=1401.24
Output text lengths: min=27, max=404, mean=187.67
Sample 0:
 

In [ ]:
def verify_encodings(encodings, name):
    expected_keys = ['input_ids', 'attention_mask', 'labels']
    print(f"\nVerifying {name} encodings:")
    print("Keys:", list(encodings.keys()))
    if not all(key in encodings for key in expected_keys):
        raise ValueError(f"{name} encodings missing expected keys: {expected_keys}")
    for key in expected_keys:
        print(f"{key} type:", type(encodings[key]))

def check_shapes(encodings, name, max_length=512):
    print(f"\n{name} encodings shapes:")
    for key in ['input_ids', 'attention_mask', 'labels']:
        shape = encodings[key].shape
        print(f"{key}: {shape}")
        if len(shape) != 2 or shape[1] != max_length:
            raise ValueError(f"{name} {key} has incorrect shape: {shape}")

def decode_sample(encodings, tokenizer, index=0):
    print(f"\nDecoding {index} from {name}:")
    input_text = tokenizer.decode(encodings['input_ids'][index], skip_special_tokens=False)
    label_text = tokenizer.decode(encodings['labels'][index], skip_special_tokens=False)
    print("Input text (decoded):", input_text[:100], "...")  # Truncate for readability
    print("Label text (decoded):", label_text[:100], "...")

def check_token_ids(encodings, tokenizer, name):
    print(f"\n{name} token ID checks:")
    for key in ['input_ids', 'labels']:
        min_id = encodings[key].min().item()
        max_id = encodings[key].max().item()
        vocab_size = tokenizer.vocab_size
        print(f"{key} min ID: {min_id}, max ID: {max_id}, vocab size: {vocab_size}")
        if min_id < 0 or max_id >= vocab_size:
            raise ValueError(f"{name} {key} contains invalid token IDs")
    unique_mask_values = encodings['attention_mask'].unique()
    print("Attention mask unique values:", unique_mask_values.tolist())

# Run all checks for each dataset
for encodings, name in [(train_encodings, "Train"), (val_encodings, "Validation"), (test_encodings, "Test")]:
    verify_encodings(encodings, name)
    check_shapes(encodings, name)
    decode_sample(encodings, tokenizer, index=0)
    check_token_ids(encodings, tokenizer, name)

# Compare with DataFrame (optional)
print("\nOriginal DataFrame sample (Train):")
print("input_text:", train_df['input_text'].iloc[0][:100], "...")
print("output_text:", train_df['output_text'].iloc[0][:100], "...")


Verifying Train encodings:
Keys: ['input_ids', 'attention_mask', 'labels', 'labels_attention_mask']
input_ids type: <class 'torch.Tensor'>
attention_mask type: <class 'torch.Tensor'>
labels type: <class 'torch.Tensor'>

Train encodings shapes:
input_ids: torch.Size([5000, 512])
attention_mask: torch.Size([5000, 512])
labels: torch.Size([5000, 512])

Decoding 0 from Train:
Input text (decoded): Summarize: SECTION 1. SHORT TITLE. This Act may be cited as the <unk> California Coastal Protection  ...
Label text (decoded): California Coastal Protection and Louisiana Energy Enhancement Act - Directs the Secretary of the In ...

Train token ID checks:
input_ids min ID: 1, max ID: 31995, vocab size: 32000
labels min ID: 1, max ID: 31995, vocab size: 32000
Attention mask unique values: [0, 1]

Verifying Validation encodings:
Keys: ['input_ids', 'attention_mask', 'labels', 'labels_attention_mask']
input_ids type: <class 'torch.Tensor'>
attention_mask type: <class 'torch.Tensor'>
labels type: <c

In [ ]:
from torch.utils.data import Dataset

class SummarizationDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __len__(self):
        return len(self.encodings['input_ids'])
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item.pop('labels_attention_mask', None)
        return item

train_dataset = SummarizationDataset(train_encodings)
val_dataset = SummarizationDataset(val_encodings)
test_dataset = SummarizationDataset(test_encodings)

print("Train dataset size:", len(train_dataset))
print("Validation dataset size:", len(val_dataset))
print("Test dataset size:", len(test_dataset))

Train dataset size: 5000
Validation dataset size: 437
Test dataset size: 1759


In [ ]:
from transformers import AutoModelForSeq2SeqLM
from peft import get_peft_model, LoraConfig
import torch

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.1,
    task_type="SEQ_2_SEQ_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

lora_adapter_path = "./lora_adapters/summarization"
model.save_pretrained(lora_adapter_path)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

trainable params: 1,769,472 || all params: 224,673,024 || trainable%: 0.7876


In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq

training_args = TrainingArguments(
    output_dir="./lora_training",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    eval_strategy="steps",
    eval_steps=500,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    logging_steps=100,
    load_best_model_at_end=True,
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator
)

trainer.train()
trainer.model.save_pretrained("./lora_adapters/summarization_trained")

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
500,0.843700,1.350701
1000,0.787500,1.324559
1500,0.740300,1.309281
2000,0.787800,1.314504
2500,0.771100,1.306674
3000,0.750200,1.303535
3500,0.802200,1.299468


In [ ]:
!pip install evaluate rouge_score -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.8 MB/s eta 0:00:00


**First load the base model i.e. T5-base using AutoModelForSeq2SeqLM.from_pretrained**

In [ ]:
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

In [ ]:
import torch
import os
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq
from peft import PeftModel
from evaluate import load
from torch.utils.data import DataLoader

**then load ur adapters and attach it with the base model, then you will be able to use the fine tuned model for evaluation.**

In [ ]:
lora_adapter_path = "/content/legal_adapters"
if not os.path.exists(lora_adapter_path):
    raise FileNotFoundError(f"LoRA adapter not found at {lora_adapter_path}")
model = PeftModel.from_pretrained(base_model, lora_adapter_path) #this line is used to attach the trained lora adpaters to the base model , so that when you evaluate its the finetuned model being used.

In [ ]:
# Verify LoRA adapter
print("Model type:", type(model))
print("Active adapters:", model.active_adapters)
# model.print_trainable_parameters()
print("LoRA config:", model.peft_config)


Model type: <class 'peft.peft_model.PeftModelForSeq2SeqLM'>
Active adapters: ['default']
LoRA config: {'default': LoraConfig(task_type='SEQ_2_SEQ_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='t5-base', revision=None, inference_mode=True, r=16, target_modules={'v', 'q'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, eva_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)}


In [ ]:
torch.cuda.empty_cache()

In [ ]:
from torch.utils.data import Subset
import random
# Load ROUGE metric
rouge = load("rouge")

# Manual batch prediction to avoid OOM

import random

subset_size = 100
indices = random.sample(range(len(test_dataset)), subset_size)
test_subset = Subset(test_dataset, indices)


batch_size = 8  # Reduced batch size
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
test_dataloader = DataLoader(test_subset, batch_size=batch_size, collate_fn=data_collator)

decoded_preds = []
decoded_labels = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for batch in test_dataloader:
    # Move batch to GPU
    batch = {k: v.to(device) for k, v in batch.items()}

    # Generate predictions
    with torch.no_grad():
        outputs = model.generate(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            max_length=150,
            num_beams=4,
            length_penalty=1.0
        )

    # Decode predictions and labels
    batch_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    batch_labels = tokenizer.batch_decode(batch["labels"], skip_special_tokens=True)

    decoded_preds.extend(batch_preds)
    decoded_labels.extend(batch_labels)

    # Clear batch tensors
    del batch, outputs, batch_preds, batch_labels
    torch.cuda.empty_cache()

# Compute ROUGE scores
rouge_results = rouge.compute(predictions=decoded_preds, references=decoded_labels)
print("ROUGE Scores:", rouge_results)

# # Compute evaluation loss (optional, for comparison with trainer.evaluate)
# eval_loss = trainer.evaluate(test_dataset)
# print("Test Results:", eval_loss)

# Print sample prediction for qualitative check
print("Sample prediction:", decoded_preds[0])
print("Sample reference:", decoded_labels[0])

# Save evaluation results
with open("summarization_eval_results.txt", "w") as f:
    # f.write("Test Results:\n")
    # f.write(str(eval_loss) + "\n\n")
    f.write("ROUGE Scores:\n")
    f.write(str(rouge_results) + "\n\n")
    f.write("Sample Prediction:\n")
    f.write(decoded_preds[0] + "\n\n")
    f.write("Sample Reference:\n")
    f.write(decoded_labels[0])

print("Evaluation results saved to: summarization_eval_results.txt")

ROUGE Scores: {'rouge1': np.float64(0.4449921555238877), 'rouge2': np.float64(0.27136629208745666), 'rougeL': np.float64(0.35846629528125773), 'rougeLsum': np.float64(0.3588330924323655)}
Sample prediction: Workplace Religious Freedom Act of 1999 - Amends the Civil Rights Act of 1964 to require an employer to: (1) consider an accommodation requiring significant difficulty or expense if it will result in the inability of an employee to perform the essential functions of the employment position of the employee; and (2) consider the identifiable cost of the accommodation, including the costs of loss of productivity and of retraining or hiring employees or transferring employees from one facility to another, in relation to the size and operating cost of the employer; and (3) determine the degree to which the geographic separation or administrative or fiscal relationship of the facilities will make the accommodation more difficult or expensive; and (4) consider the degree to which the geogr

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print("Model type:", type(trainer.model))
print("Model config:", trainer.model.peft_config)
print("Active adapters:", trainer.model.active_adapters)

Model type: <class 'peft.peft_model.PeftModelForSeq2SeqLM'>
Model config: {'default': LoraConfig(task_type='SEQ_2_SEQ_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='t5-base', revision=None, inference_mode=False, r=16, target_modules={'v', 'q'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, eva_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)}
Active adapters: ['default']


In [ ]:
def generate_summary(text, model, tokenizer, max_length=512, num_beams=4):
    inputs = tokenizer(
        f"Summarize: {text}",
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='pt'
    ).to(model.device)
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=150,
        num_beams=num_beams,
        length_penalty=1.0
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test on first 3 test samples
for i in range(3):
    input_text = test_df['input_text'].iloc[i]
    true_summary = test_df['output_text'].iloc[i]
    pred_summary = generate_summary(input_text, model, tokenizer)
    print(f"\nSample {i}:")
    print(f"Input (first 100 chars): {input_text[:100]} ...")
    print(f"True Summary: {true_summary}")
    print(f"Predicted Summary: {pred_summary}")


Sample 0:
Input (first 100 chars): [CLS] Summarize: That this Act may be cited as the ``Federal Forage Fee Act of 1993''. SECTION 1. FI ...
True Summary: Federal Forage Fee Act of 1993 - Subjects grazing operations on Federal land to applicable Federal, State, and local environmental and land use requirements. Sets forth a forage fee formula for lands under the jurisdiction of the Department of Agriculture and the Department of the Interior. [EOS]
Predicted Summary: Federal Forage Fee Act of 1993 - Requires all grazing operations conducted on any Federal lands to be subject to all applicable Federal, State, and local laws, including but not limited to: (1) Animal Damage Control Act; (2) Bankhead-Jones Farm Tenant Act; (3) Clean Air Act; (4) Endangered Species Act of 1973; (5) Federal Advisory Committee Act; (6) Insecticide, Fungicide, and Rodenticide Act; (7) Insecticide, Fungicide, and Rodenticide Act; and (8) Water Pollution Control Act; and (8) Water Pollution Control Act; and (8) 

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()


'/device:GPU:0'

# **BIOMEDICAL QA** #


No need of running legal to run this. 2 separate adapters all together

In [ ]:
!pip install -q transformers datasets peft accelerate bitsandbytes


In [ ]:
from datasets import load_dataset
import pandas as pd
import torch
import re
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType




Load & Peek PubMedQA

In [ ]:
# “pqa_labeled” split provides train/validation/test
ds = load_dataset("pubmed_qa", "pqa_labeled")

print(ds)
print(ds["train"][0])


# ✅ 3. Convert to DataFrame and create splits
df = pd.DataFrame(ds["train"])


In [ ]:
from datasets import load_dataset
import pandas as pd

# Replace the original load line
ds = load_dataset("pubmed_qa", "pqa_artificial")

# Convert to DataFrame and split
df = pd.DataFrame(ds["train"])
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
total_size = len(df)
val_size = int(0.03 * total_size)  # ~21,200 examples
test_size = int(0.03 * total_size)  # ~21,200 examples
train_size = total_size - val_size - test_size  # ~169,600 examples

train_df = df[:train_size].copy()
val_df = df[train_size:train_size + val_size].copy()
test_df = df[train_size + val_size:].copy()

print("Columns:", train_df.columns)  # Expected: ['pubid', 'question', 'context', 'long_answer', 'final_decision']
print("Sample:\n", train_df[['question', 'context', 'final_decision']].head(1))

# Continue with your existing preprocessing (clean_text, clean_context, format_for_qa, etc.)

Columns: Index(['pubid', 'question', 'context', 'long_answer', 'final_decision'], dtype='object')
Sample:
                                             question  \
0  Do the Norwegian national project for ethics s...   

                                             context final_decision  
0  {'contexts': ['Internationally, clinical ethic...            yes  


In [ ]:
#split intro train test

In [ ]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
total_size = len(df)
val_size = int(0.1 * total_size)  # ~100 examples
test_size = int(0.1 * total_size)  # ~100 examples
train_size = total_size - val_size - test_size  # ~800 examples

train_df = df[:train_size].copy()
val_df = df[train_size:train_size + val_size].copy()
test_df = df[train_size + val_size:].copy()

In [ ]:
print("Columns:", train_df.columns)  # Expected: ['pubid', 'question', 'context', 'long_answer', 'final_decision']
print("Sample:\n", train_df[['question', 'context', 'final_decision']].head(1))

Columns: Index(['pubid', 'question', 'context', 'long_answer', 'final_decision'], dtype='object')
Sample:
                                             question  \
0  Is anomalous origin of the right coronary arte...   

                                             context final_decision  
0  {'contexts': ['Anomalous origin of the right c...             no  


 Preprocess → input_text / target_text

In [ ]:
def clean_text(text):
    """Clean question, context, or answer."""
    if not isinstance(text, str):
        return ""
    # Remove HTML tags, special characters, and normalize whitespace
    text = re.sub(r'<[^>]+>', '', text)  # Remove HTML
    text = re.sub(r'[^\w\s.,!?]', '', text)  # Remove special chars
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def clean_context(context):
    """Clean and join context paragraphs from dict."""
    if not isinstance(context, dict) or 'contexts' not in context:
        return ""
    return " ".join(clean_text(c) for c in context['contexts'])

train_df['question'] = train_df['question'].apply(clean_text)
train_df['context'] = train_df['context'].apply(clean_context)
train_df['final_decision'] = train_df['final_decision'].apply(clean_text)
val_df['question'] = val_df['question'].apply(clean_text)
val_df['context'] = val_df['context'].apply(clean_context)
val_df['final_decision'] = val_df['final_decision'].apply(clean_text)
test_df['question'] = test_df['question'].apply(clean_text)
test_df['context'] = test_df['context'].apply(clean_context)
test_df['final_decision'] = test_df['final_decision'].apply(clean_text)

In [ ]:
# ✅ 6. Filter data
min_question_words = 5
min_context_words = 50
max_context_words = 1000

def word_count(text):
    """Count words in text."""
    if not isinstance(text, str) or not text:
        return 0
    return len(text.split())

# Filter based on length
train_df = train_df[
    (train_df['question'].apply(word_count) >= min_question_words) &
    (train_df['context'].apply(word_count) >= min_context_words) &
    (train_df['context'].apply(word_count) <= max_context_words) &
    (train_df['final_decision'].isin(['yes', 'no', 'maybe']))
]
val_df = val_df[
    (val_df['question'].apply(word_count) >= min_question_words) &
    (val_df['context'].apply(word_count) >= min_context_words) &
    (val_df['context'].apply(word_count) <= max_context_words) &
    (val_df['final_decision'].isin(['yes', 'no', 'maybe']))
]
test_df = test_df[
    (test_df['question'].apply(word_count) >= min_question_words) &
    (test_df['context'].apply(word_count) >= min_context_words) &
    (test_df['context'].apply(word_count) <= max_context_words) &
    (test_df['final_decision'].isin(['yes', 'no', 'maybe']))
]

# Remove duplicates and nulls
train_df = train_df.drop_duplicates(subset=['question', 'context']).dropna(subset=['question', 'context', 'final_decision'])
val_df = val_df.drop_duplicates(subset=['question', 'context']).dropna(subset=['question', 'context', 'final_decision'])
test_df = test_df.drop_duplicates(subset=['question', 'context']).dropna(subset=['question', 'context', 'final_decision'])


In [ ]:
# ✅ 7. Format for QA
def format_for_qa(row):
    """Format as input-output pair for QA, consistent with BillSum."""
    input_text = f"[CLS] Question: {row['question']} Context: {row['context']} [SEP]"
    output_text = f"Answer: {row['final_decision']} [EOS]"
    return pd.Series({'input_text': input_text, 'output_text': output_text})

train_df[['input_text', 'output_text']] = train_df.apply(format_for_qa, axis=1)
val_df[['input_text', 'output_text']] = val_df.apply(format_for_qa, axis=1)
test_df[['input_text', 'output_text']] = test_df.apply(format_for_qa, axis=1)

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer

# Define your model and tokenizer
model_name = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token

# Define max lengths
max_input_length = 256
max_output_length = 20

# Convert your DataFrames to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Define tokenization function
def tokenize_function(example):
    # Tokenize input
    inputs = tokenizer(
        example["input_text"],
        max_length=max_input_length,
        padding="max_length",
        truncation=True
    )
    # Tokenize output
    outputs = tokenizer(
        example["output_text"],
        max_length=max_output_length,
        padding="max_length",
        truncation=True
    )
    # Return combined dictionary
    inputs["labels"] = outputs["input_ids"]
    return inputs

# Apply tokenizer using map (RAM-efficient, lazy)
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=train_dataset.column_names)
val_dataset = val_dataset.map(tokenize_function, batched=True, remove_columns=val_dataset.column_names)
test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=test_dataset.column_names)


Map:   0%|          | 0/168613 [00:00<?, ? examples/s]

Map:   0%|          | 0/21078 [00:00<?, ? examples/s]

Map:   0%|          | 0/21074 [00:00<?, ? examples/s]

In [ ]:
# ✅ 9. Save processed data
# Save as CSV for inspection
train_df[['input_text', 'output_text']].to_csv("pubmedqa_train_processed.csv", index=False)
val_df[['input_text', 'output_text']].to_csv("pubmedqa_val_processed.csv", index=False)
test_df[['input_text', 'output_text']].to_csv("pubmedqa_test_processed.csv", index=False)

# Save tokenized data for training
torch.save(train_dataset, "pubmedqa_train_tokenized.pt")
torch.save(val_dataset, "pubmedqa_val_tokenized.pt")
torch.save(test_dataset, "pubmedqa_test_tokenized.pt")

# ✅ 10. Summary statistics
print(f"Train size: {len(train_df)}")
print(f"Validation size: {len(val_df)}")
print(f"Test size: {len(test_df)}")
print("Sample input-output pair:")
print(train_df[['input_text', 'output_text']].iloc[0])

Train size: 168613
Validation size: 21078
Test size: 21074
Sample input-output pair:
input_text     [CLS] Question: Is anomalous origin of the rig...
output_text                                     Answer: no [EOS]
Name: 0, dtype: object


In [ ]:
# ✅ 11. Create PyTorch Dataset (for training)
from torch.utils.data import Dataset

class QADataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __len__(self):
        return len(self.encodings['input_ids'])
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item.pop('labels_attention_mask', None)  # Not needed for T5 training
        return item

train_dataset = QADataset(train_dataset)
val_dataset = QADataset(val_dataset)
test_dataset = QADataset(test_dataset)

print("Train dataset size:", len(train_dataset))
print("Validation dataset size:", len(val_dataset))
print("Test dataset size:", len(test_dataset))


Train dataset size: 168613
Validation dataset size: 21078
Test dataset size: 21074


PREPROCESSING DONE

In [ ]:
# ✅ 7. Initialize T5 model with LoRA
# Load base T5 model
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Use 16-bit precision for efficiency
    device_map="auto"  # Automatically map to GPU if available
)

# Configure LoRA
lora_config = LoraConfig(
    task_type="SEQ_2_SEQ_LM",
    inference_mode=False,
    r=8,  # Low rank for efficiency
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q", "v"]  # T5 query/value projections
)

# Apply LoRA to the model
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

# ✅ 8. Verify dataset sizes
print("Train dataset size:", len(train_dataset))
print("Validation dataset size:", len(val_dataset))
print("Test dataset size:", len(test_dataset))

In [ ]:
#trying a different code above. the one where i asked it just for t5 tokenisation and then lora adapter
#update: no errors!

TRAINING ARGUMENTS AND TRAINER

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

output_dir = "qa_biomed_lora"
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    learning_rate=3e-4,
    warmup_steps=100,
    logging_steps=50,
    save_total_limit=2,
    fp16=True,
    report_to="none",
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, label_pad_token_id=tokenizer.pad_token_id)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()
metrics = trainer.evaluate(eval_dataset=val_dataset)
print("Validation metrics:", metrics)

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"✅ Saved adapter + tokenizer to `{output_dir}`")


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModelForSeq2SeqLM

# 1.1 Reload base T5 + adapter
model_name = "t5-base"
tokenizer  = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

base = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
qa_adapter = PeftModelForSeq2SeqLM.from_pretrained(base, "qa_biomed_lora")
qa_adapter.eval()

# 1.2 Define a helper to run QA
def run_qa(question, context, max_len=32):
    prompt = f"question: {question} context: {context}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding="longest").to(qa_adapter.device)
    out = qa_adapter.generate(**inputs, max_new_tokens=max_len)
    return tokenizer.decode(out[0], skip_special_tokens=True)

# 1.3 Test a couple of examples from your held‑out test set
# for idx in range(3):
#     row = test_df.iloc[idx]
#     print(row)
#     ans = run_qa(row.question, row.context)
#     print(f"\nQ: {row.question}\n▶️ Predicted: {ans}\n✔️ Actual:    {row.final_decision}")

for idx in range(10):
    row = test_df.iloc[idx]
    prompt = row["input_text"]

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding="longest").to(qa_adapter.device)
    out = qa_adapter.generate(**inputs, max_new_tokens=32)
    pred = tokenizer.decode(out[0], skip_special_tokens=True)

    print(f"\n▶️ Prompt:    {prompt}")
    print(f"🧠 Predicted: {pred}")
    print(f"✅ Actual:    {row['output_text']}")


In [ ]:
print(test_df.columns)


sample of the dataset


In [ ]:
import pandas as pd

data = [
    {
        "question": "Is metformin used to treat type 2 diabetes?",
        "context": "Metformin is an oral antihyperglycemic agent that improves glycemic control in patients with type 2 diabetes by decreasing hepatic glucose production and increasing insulin sensitivity.",
        "final_decision": "yes"
    },
    {
        "question": "Does aspirin inhibit platelet aggregation?",
        "context": "Aspirin irreversibly inhibits cyclooxygenase-1 (COX-1) in platelets, leading to decreased formation of thromboxane A2 and reduced platelet aggregation.",
        "final_decision": "yes"
    },
    {
        "question": "Is ibuprofen a nonsteroidal anti-inflammatory drug?",
        "context": "Ibuprofen is widely used to relieve pain, reduce inflammation, and lower fever as a member of the NSAID class, acting via COX inhibition.",
        "final_decision": "yes"
    },
    {
        "question": "Does paracetamol have anti-inflammatory properties?",
        "context": "Paracetamol (acetaminophen) is used primarily for its analgesic and antipyretic effects; it has minimal peripheral anti-inflammatory action compared to NSAIDs.",
        "final_decision": "no"
    },
    {
        "question": "Can statins be used to lower cholesterol?",
        "context": "Statins inhibit HMG-CoA reductase, the rate-limiting enzyme in cholesterol biosynthesis, thereby lowering LDL cholesterol levels in the bloodstream.",
        "final_decision": "yes"
    },
    {
        "question": "Is metronidazole effective against viral infections?",
        "context": "Metronidazole is an antibacterial and antiprotozoal agent effective against anaerobic bacteria and certain protozoa, but it has no activity against viruses.",
        "final_decision": "no"
    },
    {
        "question": "Does ciprofloxacin treat urinary tract infections?",
        "context": "Ciprofloxacin is a broad-spectrum fluoroquinolone antibiotic that is commonly prescribed for urinary tract infections, among other bacterial infections.",
        "final_decision": "yes"
    },
    {
        "question": "Is oseltamivir used to treat influenza A and B?",
        "context": "Oseltamivir is a neuraminidase inhibitor indicated for treatment and prophylaxis of influenza A and B viruses in adults and pediatric patients.",
        "final_decision": "yes"
    },
    {
        "question": "Does amoxicillin cover Pseudomonas aeruginosa?",
        "context": "Amoxicillin is a broad-spectrum penicillin antibiotic but lacks reliable activity against Pseudomonas aeruginosa, which often requires antipseudomonal agents.",
        "final_decision": "no"
    },
    {
        "question": "Is heparin an anticoagulant?",
        "context": "Heparin enhances the activity of antithrombin III, leading to inactivation of thrombin and factor Xa, and is widely used as an anticoagulant in clinical settings.",
        "final_decision": "yes"
    },
    {
        "question": "Can vancomycin treat MRSA infections?",
        "context": "Vancomycin is a glycopeptide antibiotic that remains one of the primary treatments for infections caused by methicillin-resistant Staphylococcus aureus (MRSA).",
        "final_decision": "yes"
    },
    {
        "question": "Does acetazolamide treat glaucoma?",
        "context": "Acetazolamide is a carbonic anhydrase inhibitor that reduces aqueous humor production and is used to lower intraocular pressure in glaucoma patients.",
        "final_decision": "yes"
    },
    {
        "question": "Is aspirin used to treat hypertension?",
        "context": "Aspirin is primarily an antiplatelet and analgesic agent; it has no established role in lowering blood pressure for hypertension management.",
        "final_decision": "no"
    },
    {
        "question": "Does trimethoprim-sulfamethoxazole cover urinary pathogens?",
        "context": "Trimethoprim-sulfamethoxazole is a combination antibiotic commonly used to treat urinary tract infections due to its activity against E. coli and other uropathogens.",
        "final_decision": "yes"
    },
    {
        "question": "Can prednisone be used to treat autoimmune diseases?",
        "context": "Prednisone is a corticosteroid with potent anti-inflammatory and immunosuppressive effects, frequently used in autoimmune conditions like rheumatoid arthritis and lupus.",
        "final_decision": "yes"
    }
]

df = pd.DataFrame(data)
# Ensure path exists in your Kaggle notebook
output_path = "user_biomed_qa_sample.csv"
df.to_csv(output_path, index=False)
print(f"✅ Saved sample CSV to `{output_path}`")


✅ Saved sample CSV to `user_biomed_qa_sample.csv`
